# Data Preparation

In [1]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [3]:
features

,gre,gpa,rank_1,rank_2,rank_3,rank_4
106,0.972155,0.446965,1,0,0,0
9,0.972155,1.392922,0,1,0,0
61,-0.239793,-0.183673,0,0,0,1
224,1.837832,-1.287291,0,1,0,0
37,-0.586063,-1.287291,0,0,1,0
242,0.799020,-1.129631,0,0,1,0
313,-0.586063,0.683454,0,0,0,1
52,1.318426,-0.052290,0,0,0,1
347,-0.066657,1.077603,0,1,0,0
239,0.972155,0.157922,0,0,1,0


# Neural Network with one hidden layer and back propagation

In [59]:
import numpy as np

np.random.seed(21)

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5, size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5, size=n_hidden)

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # Calculate the output
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))

        ## Backward pass ##
        # Calculate the network's prediction error
        error = y - output

        # Calculate error term for the output unit
        output_error_term = error * output * (1 - output)

        ## propagate errors to hidden layer

        # Calculate the hidden layer's contribution to the error
        hidden_error = output_error_term * weights_hidden_output
        
        # Calculate the error term for the hidden layer
        hidden_error_term = hidden_error * hidden_output * (1 - hidden_output)
        
        # Update the change in weights
        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:,None]

    # Update weights
    weights_hidden_output += learnrate * del_w_hidden_output / n_records
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))


('Train loss: ', 0.25135725242598811)
('Train loss: ', 0.24996540718842905)
('Train loss: ', 0.24862005218904504)
('Train loss: ', 0.2473199321717981)
('Train loss: ', 0.24606380465584854)
('Train loss: ', 0.24485044179257037)
('Train loss: ', 0.24367863201868301)
('Train loss: ', 0.24254718151769472)
('Train loss: ', 0.24145491550165454)
('Train loss: ', 0.24040067932493334)
Prediction accuracy: 0.725
